#Project 1 - Report 1 code

In [ ]:
#Please have the 'playlists4000.txt' file in the same directory as this notebook.

fd = open("playlists4000.txt",mode='r',encoding='utf8',newline='\n')

raw = fd.read().splitlines()

hashtable = dict()

for line in raw:
    first, second = line.split(sep="::")
    hashtable.setdefault(first, []).append(second)

In [ ]:
len(hashtable.values())

In [ ]:
#Longest transaction
longest = 0
for x in hashtable.values():
    if (len(x) > longest):
        longest = len(x)

longest

In [ ]:
#Shortest transaction
shortest = longest
for x in hashtable.values():
    if(len(x) < shortest):
        shortest = len(x)
shortest

In [ ]:
#Support counting

#Dictionary of 1-itemsets with support count

counts = dict()

for line in raw:
    first, second = line.split(sep="::")
    x = counts.setdefault(second,0)
    counts[second] = x+1

In [ ]:
len(counts.values())

In [ ]:
#Number of frequent 1-itemsets with support count threshold 400
freq = 0

for x in counts:
    if(counts[x] > 40):
        freq += 1

freq

In [ ]:
#Highest support for a 1-itemset
max(counts.values())

In [ ]:
import pandas as pd

df_counts = pd.DataFrame.from_dict(counts, orient='index')

df_counts = df_counts.set_axis(['support'],axis=1)

df_counts.head()

In [ ]:
plot_df = df_counts.loc[df_counts['support'] > 50]

plot_df = plot_df.sort_values(by=['support']).reset_index()

plot_df = plot_df.rename(columns={'index' : 'Items','support':'Support'})

plot_df

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(num=1,figsize=(20,15), dpi=100)
plt.bar(plot_df['Items'], plot_df['Support'])
plt.xticks(np.arange(0, 333, 10))
plt.xlabel('Item')
plt.ylabel('Support')
plt.title('Support counts in increasing order')
plt.show()

In [ ]:
# minsup, minconf = 5, 0.5
minsup, minconf = 30, 0.7
# minsup, minconf = 60, 0.5
# minsup, minconf = 60, 0.5

In [ ]:
# FP-Tree generateion of 4-itemsets with support count and pruning
from collections import OrderedDict
fp = OrderedDict()

# 1-itemset support counting
for t in hashtable.values():
    for ii in range(len(t)):
        i = int(t[ii])
        fp[i] = fp.get(i, OrderedDict())
        fp[i]["count"] = fp[i].get("count", 0) + 1

# 2-itemset support counting
for t in hashtable.values():
    for ii in range(len(t)):
        i = int(t[ii])
        # if support count is too low
        # prune 1-itemset
        if i in fp and fp[i]["count"] < minsup:
            continue
        if i not in fp:
            continue
        for ji in range(ii+1, len(t)):
            j = int(t[ji])
            fp[i][j] = fp[i].get(j, OrderedDict())
            fp[i][j]["count"] = fp[i][j].get("count", 0) + 1

# 3-itemset support counting            
for t in hashtable.values():
    for ii in range(len(t)):
        i = int(t[ii])
        # if fp[i]["count"] < minsup:
        #     continue
        if i not in fp:
            continue
        for ji in range(ii+1, len(t)):
            j = int(t[ji])
            # if support count is too low
            # prune 2-itemset
            if j in fp[i] and fp[i][j]["count"] < minsup:
                del fp[i][j]
                continue
            if j not in fp[i]:
                continue    
            for ki in range(ji+1, len(t)):
                k = int(t[ki])
                fp[i][j][k] = fp[i][j].get(k, OrderedDict())
                fp[i][j][k]["count"] = fp[i][j][k].get("count", 0) + 1

# 4-itemset support couting
for t in hashtable.values():
    for ii in range(len(t)):
        i = int(t[ii])
        # if fp[i]["count"] < minsup:
        #     continue
        if i not in fp:
            continue
        for ji in range(ii+1, len(t)):
            j = int(t[ji])
            # if fp[i][j]["count"] < minsup:
            #     continue
            if j not in fp[i]:
                continue 
            for ki in range(ji+1, len(t)):
                k = int(t[ki])
                # if support count is too low
                # prune 3-itemset
                if k in fp[i][j] and fp[i][j][k]["count"] < minsup:
                    del fp[i][j][k]
                    continue
                if k not in fp[i][j]:
                    continue
                for li in range(ki+1, len(t)):
                    l = int(t[li])
                    fp[i][j][k][l] = fp[i][j][k].get(l, OrderedDict())
                    fp[i][j][k][l]["count"] = fp[i][j][k][l].get("count", 0) + 1
                
ctr = 0
for k, v in fp.items():
    ctr += v["count"]
fp["count"] = ctr

In [ ]:
print(fp["count"])

In [ ]:
# Support counting

# Dictionary of 1-itemsets, 2-itemsets, and 3-itemsets with support count

itemsets = dict()
itemsets[1] = OrderedDict()
itemsets[2] = OrderedDict()
itemsets[3] = OrderedDict()
itemsets[4] = OrderedDict()

def generate(node, s, len=0):
    # Support count-based pruning
    if "count" in node and node["count"] < minsup:
        return
    if len >= 1:
        itemset = tuple(sorted(s))
        itemsets[len][itemset] = node["count"]
    if len == 4:
        return
    for k in node:
        if k != "count":
            generate(node[k], s + [int(k)], len+1)

generate(fp, [])

In [ ]:
# Rule generation for 2-itemsets with minsup a and minconf b
rules = dict()
rules[2] = dict()
rules[3] = dict()
rules[2]["antecedent"] = list()
rules[2]["consequent"] = list()
rules[2]["support count"] = list()
rules[2]["confidence"] = list()
rules[3]["antecedent"] = list()
rules[3]["consequent"] = list()
rules[3]["support count"] = list()
rules[3]["confidence"] = list()


for k, v in itemsets[2].items():
    # print(", ".join(list(map(str, list(k)))), end="\t")
    # print("count: ", str(v))
    f, s = k
    n, d = v, itemsets[1][tuple([f])]
    conf = n / d
    if conf >= minconf:
        rules[2]["antecedent"].append(f)
        rules[2]["consequent"].append(s)
        rules[2]["support count"].append(n)
        rules[2]["confidence"].append(round(conf, 4))
        # rules[2][f"{f} -> {s}"] =  [round(conf, 4), n]
    d = itemsets[1][tuple([s])]
    conf = n / d
    if conf >= minconf:
        rules[2]["antecedent"].append(f)
        rules[2]["consequent"].append(s)
        rules[2]["support count"].append(n)
        rules[2]["confidence"].append(round(conf, 4))
        # rules[2][f"{f} -> {s}"] = [round(conf, 4), n]

# for k, v in rules[2].items():
#     print(k, end="\t\t")
#     print("support count: ", str(v[1]), end="\t\t")
#     print("confidence: ", str(v[0])) 

In [ ]:
# Rule generation for 3-itemsets with minsup a and minconf b

seen = set()

def rule_gen_3_2(three: tuple, prevant: tuple):
    f, m, e = three
    pf, ps = prevant
    diff = list(set([f, m, e]).difference(set([pf, ps])))[0]
    ant, cons =  tuple([pf]), tuple(sorted([ps, diff]))
    n, d = itemsets[3][three], itemsets[1][ant]
    conf = n / d
    sant = ", ".join(list(map(str, list(ant))))
    scons = ", ".join(list(map(str, list(cons))))
    rulestr = f"{sant}\t--->\t{scons}"
    if conf >= minconf and rulestr not in seen:
        seen.add(rulestr)
        rules[3]["antecedent"].append(sant)
        rules[3]["consequent"].append(scons)
        rules[3]["support count"].append(n)
        rules[3]["confidence"].append(round(conf, 4))
        # rules[3][rulestr] = [round(conf, 4), n]
    ant, cons =  tuple([ps]), tuple(sorted([pf, diff]))
    n, d = itemsets[3][three], itemsets[1][ant]
    conf = n / d
    sant = ", ".join(list(map(str, list(ant))))
    scons = ", ".join(list(map(str, list(cons))))
    rulestr = f"{sant}\t--->\t{scons}"
    if conf >= minconf and rulestr not in seen:
        seen.add(rulestr)
        rules[3]["antecedent"].append(sant)
        rules[3]["consequent"].append(scons)
        rules[3]["support count"].append(n)
        rules[3]["confidence"].append(round(conf, 4))
        # rules[3][rulestr] = [round(conf, 4), n]
    

def rule_gen_3(three: tuple):
    f, m, e = three
    ant = tuple(sorted([m, e]))
    cons  = tuple([f])
    n, d = itemsets[3][three], itemsets[2][ant]
    conf = n / d
    sant = ", ".join(list(map(str, list(ant))))
    scons = ", ".join(list(map(str, list(cons))))
    rulestr = f"{sant}\t--->\t{scons}"
    if conf >= minconf and rulestr not in seen:
        rules[3]["antecedent"].append(sant)
        rules[3]["consequent"].append(scons)
        rules[3]["support count"].append(n)
        rules[3]["confidence"].append(round(conf, 4))
        # rules[3][f"{sant}\t--->\t{scons}"] = [round(conf, 4), n]
        rule_gen_3_2(three, ant)
    ant, cons = tuple(sorted([f, e])), tuple([m])
    n, d = itemsets[3][three], itemsets[2][ant]
    conf = n / d
    sant = ", ".join(list(map(str, list(ant))))
    scons = ", ".join(list(map(str, list(cons))))
    rulestr = f"{sant}\t--->\t{scons}"
    if conf >= minconf and rulestr not in seen:
        rules[3]["antecedent"].append(sant)
        rules[3]["consequent"].append(scons)
        rules[3]["support count"].append(n)
        rules[3]["confidence"].append(round(conf, 4))
        # rules[3][f"{sant}\t--->\t{scons}"] = [round(conf, 4), n]
        rule_gen_3_2(three, ant)
    ant, cons = tuple(sorted([f, m])), tuple([e])
    n, d = itemsets[3][three], itemsets[2][ant]
    conf = n / d
    if conf >= minconf:
        sant = ", ".join(list(map(str, list(ant))))
        scons = ", ".join(list(map(str, list(cons))))
        rules[3]["antecedent"].append(sant)
        rules[3]["consequent"].append(scons)
        rules[3]["support count"].append(n)
        rules[3]["confidence"].append(round(conf, 4))
        # rules[3][f"{sant}\t--->\t{scons}"] = [round(conf, 4), n]
        rule_gen_3_2(three, ant)
    
    
for k, v in itemsets[3].items(): 
    rule_gen_3(k)

# for k, v in rules[3].items():
#     print(k, end="\t\t")
#     print("support count: ", str(v[1]), end="\t\t")
#     print("confidence: ", str(v[0]))   

In [ ]:
#Dataframe for 2-itemset rules
import pandas as pd

two_itemset_rules = pd.DataFrame(rules[2])

In [ ]:
two_itemset_rules.head(10)

In [ ]:
#Dataframe for 3-itemset rules
import pandas as pd

three_itemset_rules = pd.DataFrame(rules[3])

In [ ]:
three_itemset_rules.head(10)